In [ ]:
#Comparing auto and JTT models in tree-puzzle for datasets

In [5]:
#Imports
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import os
import random
import glob
import numpy as np
import seaborn as sns
from scipy import stats
import subprocess
import pdb

In [1]:
def parse_puzzle(indir, key):
    '''Parse output from tree-puzzle and write to dict
    '''
    dist_file = open(indir + key + '.phy.dist')
    
    for line in dist_file:
        line = line.rstrip()
        line = line.split(" ") #split on double space
        line = list(filter(None, line)) #Filter away empty strings

        if len(line)>2:
            seq_dist = line[-1] #Get ML evolutionary distance between sequences
            break
    dist_file.close()

    return seq_dist

In [63]:
complete_dssp_df = pd.read_csv('/hdd/pbryant/results/CATH/20190904/complete_dssp_df.csv')
complete_jtt_df = pd.read_csv('/hdd/pbryant/results/CATH/20190904/complete_dssp_df_jtt.csv')

In [23]:
#Parse puzzle
uid1 = [*complete_dssp_df['uid1']]
uid2 = [*complete_dssp_df['uid2']]
hgroups = [*complete_dssp_df['H_group']]

#Get JTT evdists
base = '/hdd/pbryant/results/CATH/20190904/rerun_puzzle/AA20/JTT'
suffices = ['sequence', 'structure']
JTT_evdists = []
for i in range(len(suffices)):
    suffix = suffices[i]
    evdists = [] #Save evdists
    for i in range(len(uid1)):
        key = uid1[i]+'_'+uid2[i]
        indir = base + '/' + suffix + '/'+hgroups[i]+'/'

        seq_dist = parse_puzzle(indir, key)
        evdists.append(seq_dist)
    JTT_evdists.append(evdists)
        
        
#Get AUTO evdists
base = '/hdd/pbryant/results/CATH/20190904/rerun_puzzle/AA20/Auto'
suffices = ['sequence', 'structure']
AUTO_evdists = []
failed_groups = []
for i in range(len(suffices)):
    suffix = suffices[i]
    evdists = []
    for i in range(len(uid1)):
        key = uid1[i]+'_'+uid2[i]
        indir = base + '/' + suffix + '/'+hgroups[i]+'/'
        try:
            seq_dist = parse_puzzle(indir, key)
            evdists.append(seq_dist)
        except:
            if hgroups[i] not in failed_groups:
                print(hgroups[i])
                failed_groups.append(hgroups[i])
    AUTO_evdists.append(evdists)



In [25]:
print(len(JTT_evdists[0]), len(JTT_evdists[1]))
print(len(AUTO_evdists[0]), len(AUTO_evdists[1]))

49103 49103
49103 49103


In [72]:
#Change jtt df into correct.
x = []
[x.append(float(JTT_evdists[0][i])) for i in range(len(JTT_evdists[0]))]
complete_jtt_df['MLAAdist_seqaln'] = x

x = []
[x.append(float(JTT_evdists[1][i])) for i in range(len(JTT_evdists[1]))]
complete_jtt_df['MLAAdist_straln'] = x

In [74]:
complete_jtt_df.to_csv('/hdd/pbryant/results/CATH/20190904/complete_dssp_df_jtt.csv')